In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import random
from time import sleep
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import numpy as np
import nltk
import os
from sklearn import datasets

In [2]:
# This is needed to authenticate us to
import os
%load_ext dotenv
%dotenv
try:
    from requests_oauthlib import OAuth2
except ModuleNotFoundError:
    import sys
    import os

    # I need this because requests_oauth gets installed in a weird place on my system
    sys.path.append('/usr/local/lib/python3.6/site-packages')
    from requests_oauthlib import OAuth2

In [3]:
OAuth2

requests_oauthlib.oauth2_auth.OAuth2

In [4]:
os.getenv("CLIENT_ACCESS_TOKEN"), os.getenv("CLIENT_SECRET"), os.getenv("CLIENT_ID")

('mBnDnHyEI_KGNyYv39-7efdwjJXc5cWq77fg_eDlsO8RDkYS8uu0YvOiMPS0e-FX',
 '2oVQqSrDutfqBf4b1Vv_v7EsXOURvV4zInrTcYkH1o3Ih2zJL7ibUy3DnjG08L_egKNqJnlgxgjZwwR9AOLwDw',
 'tg16sSXJ--cefig_zx6WGTiAizP7C2gCxP3ss_51lY5IiK_tOs_yhYlGT6jh5XvX')

In [5]:
# %load ~/anaconda3/lib/python3.6/site-packages/requests_oauthlib/oauth2_auth.py
from __future__ import unicode_literals
from oauthlib.oauth2 import WebApplicationClient, InsecureTransportError
from oauthlib.oauth2 import is_secure_transport
from requests.auth import AuthBase


class OAuth2(AuthBase):
    """Adds proof of authorization (OAuth2 token) to the request."""

    def __init__(self, client_id=None, client=None, token=None):
        """Construct a new OAuth 2 authorization object.

        :param client_id: Client id obtained during registration
        :param client: :class:`oauthlib.oauth2.Client` to be used. Default is
                       WebApplicationClient which is useful for any
                       hosted application but not mobile or desktop.
        :param token: Token dictionary, must include access_token
                      and token_type.
        """
        self._client = client or WebApplicationClient(client_id, token=token)
        if token:
            for k, v in token.items():
                setattr(self._client, k, v)

    def __call__(self, r):
        """Append an OAuth 2 token to the request.

        Note that currently HTTPS is required for all requests. There may be
        a token type that allows for plain HTTP in the future and then this
        should be updated to allow plain HTTP on a white list basis.
        """
        if not is_secure_transport(r.url):
            raise InsecureTransportError()
        r.url, r.headers, r.body = self._client.add_token(r.url,
                http_method=r.method, body=r.body, headers=r.headers)
        return r


In [6]:
#%load_ext dotenv
#%dotenv
import os

token = {
    'access_token': os.getenv("CLIENT_ACCESS_TOKEN"),
    'token_type': 'Bearer'
}

oauth = OAuth2(os.getenv("CLIENT_ID"),
               None,token)

In [7]:
token

{'access_token': 'mBnDnHyEI_KGNyYv39-7efdwjJXc5cWq77fg_eDlsO8RDkYS8uu0YvOiMPS0e-FX',
 'token_type': 'Bearer'}

In [8]:
from pymongo import MongoClient
import requests

client = MongoClient()

In [9]:
client.database_names()

['admin',
 'books',
 'catalog',
 'config',
 'events',
 'gambino',
 'legistlation',
 'local']

In [10]:
db = client.gambino
songs = db.songs

In [11]:
def childish_api_to_mongo(pages):
    for i in range(1, pages+1):
        response = requests.get(("https://api.genius.com/artists/1745/songs?sort=popularity&per_page=50&page=" + str(i)),
                        auth=oauth)
        songs_json = response.json()
        songs = songs_json['response']['songs']
        db.songs.insert_many(songs)
        print('Page ' + str(i) + ' complete!')

In [32]:

db.songs.count()

0

In [33]:
# childish_api_to_mongo(9)

Page 1 complete!
Page 2 complete!
Page 3 complete!
Page 4 complete!
Page 5 complete!
Page 6 complete!
Page 7 complete!
Page 8 complete!
Page 9 complete!


In [14]:
# This "user_timeline" url tells the API to get the tweets of the user associated with the app.
response = requests.get("https://api.genius.com/artists/1745/songs?sort=popularity",
                        auth=oauth)

# Note that we're using the "requests" library
cg_songs = response.json()

In [15]:
## "https://api.genius.com/artists/1745"

len(cg_songs)

2

In [16]:
cg_songs

{'meta': {'status': 200},
 'response': {'songs': [{'annotation_count': 20,
    'api_path': '/songs/3688791',
    'full_title': 'This Is America by\xa0Childish\xa0Gambino',
    'header_image_thumbnail_url': 'https://images.genius.com/25d40b3685e26eed3e943476a29d1763.300x100x1.jpg',
    'header_image_url': 'https://images.genius.com/25d40b3685e26eed3e943476a29d1763.1000x333x1.jpg',
    'id': 3688791,
    'lyrics_owner_id': 104344,
    'lyrics_state': 'complete',
    'path': '/Childish-gambino-this-is-america-lyrics',
    'pyongs_count': 381,
    'song_art_image_thumbnail_url': 'https://images.genius.com/04cc504b70ee9a45062f125275856a4c.300x300x1.jpg',
    'stats': {'hot': False,
     'unreviewed_annotations': 0,
     'concurrents': 9,
     'pageviews': 4248761},
    'title': 'This Is America',
    'title_with_featured': 'This Is America',
    'url': 'https://genius.com/Childish-gambino-this-is-america-lyrics',
    'primary_artist': {'api_path': '/artists/1745',
     'header_image_url': '

In [17]:
first_songs = cg_songs['response']['songs']

first_songs

[{'annotation_count': 20,
  'api_path': '/songs/3688791',
  'full_title': 'This Is America by\xa0Childish\xa0Gambino',
  'header_image_thumbnail_url': 'https://images.genius.com/25d40b3685e26eed3e943476a29d1763.300x100x1.jpg',
  'header_image_url': 'https://images.genius.com/25d40b3685e26eed3e943476a29d1763.1000x333x1.jpg',
  'id': 3688791,
  'lyrics_owner_id': 104344,
  'lyrics_state': 'complete',
  'path': '/Childish-gambino-this-is-america-lyrics',
  'pyongs_count': 381,
  'song_art_image_thumbnail_url': 'https://images.genius.com/04cc504b70ee9a45062f125275856a4c.300x300x1.jpg',
  'stats': {'hot': False,
   'unreviewed_annotations': 0,
   'concurrents': 9,
   'pageviews': 4248761},
  'title': 'This Is America',
  'title_with_featured': 'This Is America',
  'url': 'https://genius.com/Childish-gambino-this-is-america-lyrics',
  'primary_artist': {'api_path': '/artists/1745',
   'header_image_url': 'https://images.genius.com/c2c84d21944bce3835b257b1fc400833.1000x210x1.jpg',
   'id': 17

In [18]:
len(first_songs)

20

In [19]:
# This "user_timeline" url tells the API to get the tweets of the user associated with the app.
response = requests.get("https://api.genius.com/artists/1745/songs?sort=popularity&per_page=20&page=2",
                        auth=oauth)

# Note that we're using the "requests" library
cg_songs2 = response.json()

In [20]:
len(cg_songs2)

2

In [21]:
second_songs = cg_songs2['response']['songs']

print(len(second_songs))
second_songs

20


[{'annotation_count': 31,
  'api_path': '/songs/57158',
  'full_title': 'L.E.S. by\xa0Childish\xa0Gambino',
  'header_image_thumbnail_url': 'https://images.genius.com/91362788ed6088d22ac72895642b3e50.300x300x1.jpg',
  'header_image_url': 'https://images.genius.com/91362788ed6088d22ac72895642b3e50.1000x1000x1.jpg',
  'id': 57158,
  'lyrics_owner_id': 11524,
  'lyrics_state': 'complete',
  'path': '/Childish-gambino-les-lyrics',
  'pyongs_count': 76,
  'song_art_image_thumbnail_url': 'https://images.genius.com/00196af444633a4108e6dc544358985e.300x300x1.jpg',
  'stats': {'hot': False, 'unreviewed_annotations': 0, 'pageviews': 381353},
  'title': 'L.E.S.',
  'title_with_featured': 'L.E.S.',
  'url': 'https://genius.com/Childish-gambino-les-lyrics',
  'primary_artist': {'api_path': '/artists/1745',
   'header_image_url': 'https://images.genius.com/c2c84d21944bce3835b257b1fc400833.1000x210x1.jpg',
   'id': 1745,
   'image_url': 'https://images.genius.com/5a9c598da4e86fefcb845cbbe74706aa.1000

In [22]:
# This "user_timeline" url tells the API to get the tweets of the user associated with the app.
response = requests.get("https://api.genius.com/artists/1745/songs?sort=popularity&per_page=20&page=3",
                        auth=oauth)

# Note that we're using the "requests" library
cg_songs3 = response.json()

In [23]:
third_songs = cg_songs3['response']['songs']

print(len(third_songs))
third_songs

20


[{'annotation_count': 45,
  'api_path': '/songs/506780',
  'full_title': 'Driving Ms. Daisy by\xa0Logic (Ft.\xa0Childish\xa0Gambino)',
  'header_image_thumbnail_url': 'https://images.genius.com/47ab70be3f64e7061a5d1f1a590e60ff.220x220x1.jpg',
  'header_image_url': 'https://images.genius.com/47ab70be3f64e7061a5d1f1a590e60ff.220x220x1.jpg',
  'id': 506780,
  'lyrics_owner_id': 282633,
  'lyrics_state': 'complete',
  'path': '/Logic-driving-ms-daisy-lyrics',
  'pyongs_count': 136,
  'song_art_image_thumbnail_url': 'https://images.genius.com/2c9956b0499470e6d5e3e5d1e65f0713.300x300x1.jpg',
  'stats': {'hot': False, 'unreviewed_annotations': 1, 'pageviews': 215216},
  'title': 'Driving Ms. Daisy',
  'title_with_featured': 'Driving Ms. Daisy (Ft.\xa0Childish\xa0Gambino)',
  'url': 'https://genius.com/Logic-driving-ms-daisy-lyrics',
  'primary_artist': {'api_path': '/artists/7922',
   'header_image_url': 'https://images.genius.com/f6b08bf607b94a279d743e14bbca2a3a.1000x563x1.jpg',
   'id': 792

In [24]:
third_songs

[{'annotation_count': 45,
  'api_path': '/songs/506780',
  'full_title': 'Driving Ms. Daisy by\xa0Logic (Ft.\xa0Childish\xa0Gambino)',
  'header_image_thumbnail_url': 'https://images.genius.com/47ab70be3f64e7061a5d1f1a590e60ff.220x220x1.jpg',
  'header_image_url': 'https://images.genius.com/47ab70be3f64e7061a5d1f1a590e60ff.220x220x1.jpg',
  'id': 506780,
  'lyrics_owner_id': 282633,
  'lyrics_state': 'complete',
  'path': '/Logic-driving-ms-daisy-lyrics',
  'pyongs_count': 136,
  'song_art_image_thumbnail_url': 'https://images.genius.com/2c9956b0499470e6d5e3e5d1e65f0713.300x300x1.jpg',
  'stats': {'hot': False, 'unreviewed_annotations': 1, 'pageviews': 215216},
  'title': 'Driving Ms. Daisy',
  'title_with_featured': 'Driving Ms. Daisy (Ft.\xa0Childish\xa0Gambino)',
  'url': 'https://genius.com/Logic-driving-ms-daisy-lyrics',
  'primary_artist': {'api_path': '/artists/7922',
   'header_image_url': 'https://images.genius.com/f6b08bf607b94a279d743e14bbca2a3a.1000x563x1.jpg',
   'id': 792

In [25]:
# ONLY RUN THIS WHEN NEW SONGS ARE ADDED
# db.songs.insert_many(first_songs)
# db.songs.insert_many(second_songs)
# db.songs.insert_many(third_songs)

In [26]:
db.songs.count()

4300

In [27]:
song_list = []

for item in db.songs.find():
    item = (item['primary_artist']['name'])
    print(item)

Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Chance The Rapper
Childish Gambino
Childish Gambino
Childish Gambino
Jhené Aiko
Childish Gambino
Kendrick Lamar
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Chance The Rapper
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Logic
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Iggy Azalea
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Ch

Papoose
Glasses Malone
Kendrick Lamar
Kendrick Lamar
Kendrick Lamar
Actual Proof
Genius Traductions Françaises
Kendrick Lamar
Kendrick Lamar & Jay Rock
Black Hippy
Kendrick Lamar
Kendrick Lamar
RGPolska Tłumaczenia
ScHoolboy Q
NJV
XXL
Kendrick Lamar
Rapper Big Pooh
Kendrick Lamar
Kendrick Lamar
Kendrick Lamar & Jay Rock
Kendrick Lamar
Gunplay
DJ Chop Up
Kanye West
Kendrick Lamar
Kendrick Lamar & Jay Rock
Kendrick Lamar
Kendrick Lamar
Kendrick Lamar
Kendrick Lamar
Kendrick Lamar & Jay Rock
Kendrick Lamar
Funkadelic
Spotify
Kendrick Lamar
Kendrick Lamar
Kendrick Lamar
Scoe
Ras Kass
Kendrick Lamar
Kendrick Lamar & Jay Rock
Kendrick Lamar & Jay Rock
Melo Kan
Kendrick Lamar
Maroon 5
Kendrick Lamar
Kendrick Lamar
Kendrick Lamar
Kendrick Lamar
Black Hippy
Kendrick Lamar
Dr. Dre
Black Friday
Eshon Burgundy
The Chainsmokers
Mykestro
Kendrick Lamar
Kendrick Lamar
Kendrick Lamar
Kendrick Lamar
¡MAYDAY!
Kendrick Lamar
Kendrick Lamar
Slim The Mobster
Kendrick Lamar
Kendrick Lamar
Casdaluchi Ent.
Ke

Kanye West
Producer Genius
Yasiin Bey
Alicia Keys
Kanye West
Fayçal
Kanye West
Solange
The Weeknd
JYJ
JAY-Z
Kanye West
JAY-Z
Kanye West
Kanye West
Kanye West
Dead Prez
GLC
Foxy Brown
Fall Out Boy
Cassidy
John Legend
Kanye West
Kanye West
Kanye West
Kanye West
Michael Jackson
Teyana Taylor
Trick Daddy
Kanye West
Madonna
Kanye West
213
Kid Cudi
Kanye West
Crooked I
Jamie Foxx
Lil Wayne
Melbeatz
Kanye West
Chamillionaire
Shyne
Common
Kanye West
Kanye West & Drake
Kanye West
Consequence
Lupe Fiasco
Kanye West
Talib Kweli
Kanye West
Pusha-T
Kanye West
Drake
Kanye West
Kanye West
Kanye West
Lil Wayne
Kanye West
Kanye West
Kanye West
Consequence
Lloyd Banks
Janet Jackson
Kanye West
Kanye West
Kanye West
The Diplomats
Cal Chuchesta
Fashion Genius
Kid Sister
W Magazine
Kanye West
Pidhi
Bryson Tiller
Consequence
B.o.B
John Legend
2 Chainz
Kanye West
Jin
Kanye West
Common
Kanye West
Kid Cudi
Malik Yusef
Do or Die
Kanye West
Kanye West
Kanye West
Rick Ross
Kanye West
Janet Jackson
Kanye West
E-40


KeyError: 'album'

In [29]:
artist = [item['primary_artist']['name'] for item in db.songs.find()]
songs = [item['title'] for item in db.songs.find()]
urls = [item['url'] for item in db.songs.find()]

result = zip(artist, album, songs, urls)
track_list = list(result)
track_list

KeyError: 'album'

In [ ]:
for track in track_list:
    if track[0] == 'Childish Gambino':
        artist = track[0]
        url = track[2]
        print(list([artist, url]))

In [ ]:
lyrics

In [ ]:
trial_list = [('Childish Gambino', 'https://genius.com/Childish-gambino-redbone-lyrics'),
              ('Childish Gambino', 'https://genius.com/Childish-gambino-v-3005-lyrics')]

In [52]:
def track_info():
    artist = [item['primary_artist']['name'] for item in db.songs.find()]
    songs = [item['title'] for item in db.songs.find()]
    urls = [item['url'] for item in db.songs.find()]

    result = zip(artist, songs, urls)
    track_list = list(result)
    return track_list

In [53]:
track_list = track_info()

In [54]:
print(len(set(track_list)))
track_list

3823


[('Childish Gambino',
  'This Is America',
  'https://genius.com/Childish-gambino-this-is-america-lyrics'),
 ('Childish Gambino',
  'Redbone',
  'https://genius.com/Childish-gambino-redbone-lyrics'),
 ('Childish Gambino',
  'V. 3005',
  'https://genius.com/Childish-gambino-v-3005-lyrics'),
 ('Childish Gambino',
  'IV. Sweatpants',
  'https://genius.com/Childish-gambino-iv-sweatpants-lyrics'),
 ('Childish Gambino',
  'Bonfire',
  'https://genius.com/Childish-gambino-bonfire-lyrics'),
 ('Childish Gambino',
  'Heartbeat',
  'https://genius.com/Childish-gambino-heartbeat-lyrics'),
 ('Chance The Rapper',
  'Favorite Song',
  'https://genius.com/Chance-the-rapper-favorite-song-lyrics'),
 ('Childish Gambino',
  'Freaks and Geeks',
  'https://genius.com/Childish-gambino-freaks-and-geeks-lyrics'),
 ('Childish Gambino',
  'III. telegraph ave. ("Oakland" by Lloyd)',
  'https://genius.com/Childish-gambino-iii-telegraph-ave-oakland-by-lloyd-lyrics'),
 ('Childish Gambino',
  'I. The Worst Guys',
  '

In [13]:
def artist_tracks(artist_name, track_list):
    tracks = []
    for track in track_list:
        if track[0] == artist_name:
            artist = track[0]
            song_name = track[1]
            url = track[2]
            tracks.append([artist, song_name, url])
    return tracks

In [ ]:
childish_tracks = artist_tracks('Childish Gambino', track_list)

In [ ]:
len(childish_tracks)

In [ ]:
childish_tracks

In [14]:
def get_lyrics(track_list):
    all_song_lyrics = []
    artist = []
    for track in track_list:
        artist = track[0]
        url = track[1]
        source_code = requests.get(url)
        soup = BeautifulSoup(source_code.text, 'html5lib')
        lyrics = soup.find('div', class_='lyrics').text.strip()
        lyrics = lyrics.split('\n\n\n')
        lyrics = lyrics.split('\n\n')
        sleep(random.uniform(2.1, 7.5))

        for i in lyrics:
            if i[:2] != '\n':
                verse = i.split('\n')
                if verse == '':
                    song_element = ''
                    lyrics = ''
                else:
                    if verse[0][0] == '[':
                        song_element = verse[0]
                        lyrics = verse[1:]
                    else:
                        song_element = ''
                        lyrics = verse
            else:
                song_element = ''
                lyrics = ''
            all_song_lyrics.append([artist,song_element,lyrics])
            
    return all_song_lyrics

In [21]:
def scrape_lyrics(track_list):
    all_song_lyrics = []
    artist = []
    for track in track_list:
        artist = track[0]
        song_name = track[1]
        url = track[2]
        try:
            source_code = requests.get(url)
            soup = BeautifulSoup(source_code.text, 'html5lib')
            lyrics = soup.find('div', class_='lyrics').text.strip()
        except:
            lyrics =''
        all_song_lyrics.append([artist,song_name,lyrics])
        sleep(random.uniform(0.1, 1.5))
        
    return all_song_lyrics
        
def split_lyrics(lyrics):
    all_song_elements = []
    for i in lyrics:
        artist = i[0]
        song_name = i[1]
        all_lyrics = i[2]
        split_lyrics = all_lyrics.split('\n\n\n')
        split_lyrics = all_lyrics.split('\n\n')

        for i in split_lyrics:
            if i[:2] != '\n':
                verse = i.split('\n')
                if verse == '':
                    song_element = ''
                    only_lyrics = ''
                else:
                    if verse[0][0] == '[':
                        song_element = verse[0]
                        only_lyrics = verse[1:]
                    else:
                        song_element = ''
                        only_lyrics = verse
            else:
                song_element = ''
                only_lyrics = ''
            all_song_elements.append([artist,song_name,song_element,only_lyrics])

    return all_song_elements

In [ ]:
source_code = requests.get('https://genius.com/Childish-gambino-ii-shadows-lyrics')
soup = BeautifulSoup(source_code.text, 'html5lib')
lyrics = soup.find('div', class_='lyrics').text.strip()
lyrics = lyrics.split('\n\n')

lyrics

In [ ]:
all_song_lyrics = []

for i in lyrics:
    if i[:2] != '\n':
        verse = i.split('\n')
        if verse == '':
            song_element = ''
            lyrics = ''
        else:
            if verse[0][0] == '[':
                song_element = verse[0]
                lyrics = verse[1:]
            else:
                song_element = ''
                lyrics = verse
    else:
        song_element = ''
        lyrics = ''
    print(song_element, lyrics)

In [ ]:
lyrics

In [ ]:
# childish_lyrics_first5 = scrape_lyrics(childish_tracks[:5])

In [ ]:
print(len(childish_lyrics_first5))
childish_lyrics_first5

In [ ]:
childish_first5 = split_lyrics(childish_lyrics_first5)

childish_first5

In [ ]:
df = pd.DataFrame(childish_first5)

df

In [ ]:
df[3] = df[3].apply(', '.join)

In [ ]:
df[3]

In [ ]:
df

In [ ]:
# childish_lyrics = scrape_lyrics(childish_tracks)

In [ ]:
print(len(childish_lyrics))
childish_lyrics[:10]

In [ ]:
import pickle

with open('childish_gambino_real.pkl', 'wb') as picklefile:
    pickle.dump(childish_lyrics, picklefile)

In [ ]:
## fix split_lyrics function and split all of the pkl files

In [35]:
def chance_api_to_mongo(pages):
    for i in range(1, pages+1):
        response = requests.get(("https://api.genius.com/artists/16751/songs?sort=popularity&per_page=50&page=" + str(i)),
                        auth=oauth)
        songs_json = response.json()
        songs = songs_json['response']['songs']
        db.songs.insert_many(songs)
        print('Page ' + str(i) + ' complete!')

In [36]:
# chance_api_to_mongo(8)

Page 1 complete!
Page 2 complete!
Page 3 complete!
Page 4 complete!
Page 5 complete!
Page 6 complete!
Page 7 complete!
Page 8 complete!


In [ ]:
db.songs.count()

In [37]:
def kdot_api_to_mongo(pages):
    for i in range(1, pages+1):
        response = requests.get(("https://api.genius.com/artists/1421/songs?sort=popularity&per_page=50&page=" + str(i)),
                        auth=oauth)
        songs_json = response.json()
        songs = songs_json['response']['songs']
        db.songs.insert_many(songs)
        print('Page ' + str(i) + ' complete!')

In [38]:
# kdot_api_to_mongo(16)

Page 1 complete!
Page 2 complete!
Page 3 complete!
Page 4 complete!
Page 5 complete!
Page 6 complete!
Page 7 complete!
Page 8 complete!
Page 9 complete!
Page 10 complete!
Page 11 complete!
Page 12 complete!
Page 13 complete!
Page 14 complete!
Page 15 complete!
Page 16 complete!


In [ ]:
db.songs.count()

In [39]:
def drake_api_to_mongo(pages):
    for i in range(1, pages+1):
        response = requests.get(("https://api.genius.com/artists/130/songs?sort=popularity&per_page=50&page=" + str(i)),
                        auth=oauth)
        songs_json = response.json()
        songs = songs_json['response']['songs']
        db.songs.insert_many(songs)
        print('Page ' + str(i) + ' complete!')

In [40]:
# drake_api_to_mongo(23)

Page 1 complete!
Page 2 complete!
Page 3 complete!
Page 4 complete!
Page 5 complete!
Page 6 complete!
Page 7 complete!
Page 8 complete!
Page 9 complete!
Page 10 complete!
Page 11 complete!
Page 12 complete!
Page 13 complete!
Page 14 complete!
Page 15 complete!
Page 16 complete!
Page 17 complete!
Page 18 complete!
Page 19 complete!
Page 20 complete!
Page 21 complete!
Page 22 complete!
Page 23 complete!


In [ ]:
db.songs.count()

In [41]:
def kanye_api_to_mongo(pages):
    for i in range(1, pages+1):
        response = requests.get(("https://api.genius.com/artists/72/songs?sort=popularity&per_page=50&page=" + str(i)),
                        auth=oauth)
        songs_json = response.json()
        songs = songs_json['response']['songs']
        db.songs.insert_many(songs)
        print('Page ' + str(i) + ' complete!')

In [42]:
# kanye_api_to_mongo(28)

Page 1 complete!
Page 2 complete!
Page 3 complete!
Page 4 complete!
Page 5 complete!
Page 6 complete!
Page 7 complete!
Page 8 complete!
Page 9 complete!
Page 10 complete!
Page 11 complete!
Page 12 complete!
Page 13 complete!
Page 14 complete!
Page 15 complete!
Page 16 complete!
Page 17 complete!
Page 18 complete!
Page 19 complete!
Page 20 complete!
Page 21 complete!
Page 22 complete!
Page 23 complete!
Page 24 complete!
Page 25 complete!
Page 26 complete!
Page 27 complete!
Page 28 complete!


In [43]:
db.songs.count()

3899

In [44]:
track_list = track_info()

NameError: name 'track_info' is not defined

In [ ]:
print(len(set(track_list)))

In [ ]:
chance_tracks = artist_tracks('Chance The Rapper', track_list)

print(len(chance_tracks))
chance_tracks[:5]

In [19]:
kdot_tracks = artist_tracks('Kendrick Lamar', track_list)

print(len(kdot_tracks))
kdot_tracks

351


[['Kendrick Lamar',
  'Look Out For Detox',
  'https://genius.com/Kendrick-lamar-look-out-for-detox-lyrics'],
 ['Kendrick Lamar',
  'HUMBLE.',
  'https://genius.com/Kendrick-lamar-humble-lyrics'],
 ['Kendrick Lamar',
  '\u200bm.A.A.d city',
  'https://genius.com/Kendrick-lamar-maad-city-lyrics'],
 ['Kendrick Lamar',
  'Swimming Pools (Drank)',
  'https://genius.com/Kendrick-lamar-swimming-pools-drank-lyrics'],
 ['Kendrick Lamar', 'DNA.', 'https://genius.com/Kendrick-lamar-dna-lyrics'],
 ['Kendrick Lamar',
  'Money Trees',
  'https://genius.com/Kendrick-lamar-money-trees-lyrics'],
 ['Kendrick Lamar',
  "Bitch, Don't Kill My Vibe",
  'https://genius.com/Kendrick-lamar-bitch-dont-kill-my-vibe-lyrics'],
 ['Kendrick Lamar', 'XXX.', 'https://genius.com/Kendrick-lamar-xxx-lyrics'],
 ['Kendrick Lamar',
  'Poetic Justice',
  'https://genius.com/Kendrick-lamar-poetic-justice-lyrics'],
 ['Kendrick Lamar',
  'King Kunta',
  'https://genius.com/Kendrick-lamar-king-kunta-lyrics'],
 ['Kendrick Lamar'

In [ ]:
drake_tracks = artist_tracks('Drake', track_list)

print(len(drake_tracks))
drake_tracks[:5]

In [ ]:
kanye_tracks = artist_tracks('Kanye West', track_list)

print(len(kanye_tracks))
kanye_tracks[:5]

In [ ]:
# chance_lyrics = scrape_lyrics(chance_tracks)

In [ ]:
print(len(chance_lyrics))
chance_lyrics[:10]

In [ ]:
with open('chance_the_rapper.pkl', 'wb') as picklefile:
    pickle.dump(chance_lyrics, picklefile)

In [22]:
# kdot_lyrics = scrape_lyrics(kdot_tracks)

In [23]:
print(len(kdot_lyrics))
kdot_lyrics[:10]

351


[['Kendrick Lamar',
  'Look Out For Detox',
  "[Intro]\nUh, shoutout to ScHoolboy Q\nHe uglier than a motherfucker (laughs)\n\n[Verse]\nTire marks, tire marks\nFinish line with the tire marks\nWhen the relay starts I’m a runaway slave\nUgh, walking on water and running on waves\nGod MC Oh my God you gotta see\nThere’s never no I's in me\nOf an Odyssey, I’m a block away\nFire marshal's moving in\nMarshmallows inside my pen\nSweet sixteens\nGot a sweet sixteen and they deadlier than sin\nI’m so appalled\nWith the prototype with a godly protocol\nYou an amateur, they wanna pro to call\nI damage ya on camera, in Compton, in Canada\nI don’t care where ya are\nJust blink twice and I’m there where you are\nLike a shadow in the dark, you a paddle in the boat\nIn an ocean full of sharks bout to come up short\nWater in the pot, flow crack rock like Bam Bam nigga\nHave two grams nigga pay up or blam blam nigga\nHad a black Camry, bumpin' Dipset, Killa Cam nigga\nI had been around niggas, killas, 

In [25]:
import pickle

with open('kendrick.pkl', 'wb') as picklefile:
    pickle.dump(kdot_lyrics, picklefile)

In [ ]:
# drake_lyrics = scrape_lyrics(drake_tracks)

In [ ]:
print(len(drake_lyrics))
drake_lyrics[:10]

In [ ]:
with open('drake.pkl', 'wb') as picklefile:
    pickle.dump(drake_lyrics, picklefile)

In [ ]:
# kanye_lyrics = scrape_lyrics(kanye_tracks)

In [ ]:
print(len(kanye_lyrics))
kanye_lyrics[:10]

In [ ]:
with open('kanye.pkl', 'wb') as picklefile:
    pickle.dump(kanye_lyrics, picklefile)